https://stockanalysis.com/list/hong-kong-stock-exchange/

In [1]:
# scraping by hand
import pandas as pd
file_path = "test.txt"

try:
    # 假设 .txt 文件内容是以制表符分隔的
    df = pd.read_csv(file_path, delimiter="	")
    df.iloc[:, :].to_csv("hk_list.csv")
except Exception as e:
    print(e)

In [2]:
df = pd.read_csv("hk_list.csv", index_col=None)
# df.columns = ["code", "ename", "price", "% Change", "Revenue"]
df.iloc[:, 1] = df.iloc[:, 1].apply(lambda x: f"{int(float(x)):04}.HK" if pd.notnull(x) else None)
df.to_csv("hk_list.csv")

/var/folders/cb/s9sjq6dx01990v5hdvfhl7x40000gn/T/ipykernel_22588/3596059803.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       0001.HK
1       0002.HK
2       0003.HK
3       0004.HK
4       0005.HK
         ...   
2536    9995.HK
2537    9996.HK
2538    9997.HK
2539    9998.HK
2540    9999.HK
Name: Unnamed: 1, Length: 2541, dtype: object' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 1] = df.iloc[:, 1].apply(lambda x: f"{int(float(x)):04}.HK" if pd.notnull(x) else None)


In [3]:
# functions
def get_demean_weighting(factor):
    df1 = factor.dropna(axis='columns', how='all').copy()
    demean = df1.sub(df1.mean(axis=1), axis=0)
    weighting = demean.div(demean.abs().sum(axis=1), axis=0)
    return weighting
def _reweighting_equal(weighting:pd.DataFrame):
    def equal_weight(row: pd.Series):
        count_larger_than_zero = (row != 0).sum()
        if count_larger_than_zero > 0:
            row = row.apply(lambda x: 1 / count_larger_than_zero if x > 0 else x)
        return row
    return weighting.apply(equal_weight, axis=1)
def intraday_shifted_input_signal_pct_output_plot(ln_, pct_calculating_profit, quantile=10, type_='cumsum'):
        
    time_length = len(ln_)
    quantile_column = [[None]*quantile]*time_length
    quantile_df = pd.DataFrame(quantile_column)
    for da in range(time_length):
        row = ln_.iloc[da, :]
        col = row.sort_values(ascending=False).index
        for i in range(0, quantile):
            quantile_df.iloc[da, i] = list(col[i*int(len(row)/quantile):(i+1)*int(len(row)/quantile)])

    # pct_close_w_corres = pct_calculating_profit[pct_calculating_profit.columns.intersection(ln_.columns)].shift(-2)
    holdings = []
    index = ln_.index
    quantiles = [None] * quantile
    for qt_iter in range(quantile):
        quantile_1 = quantile_df.iloc[:, qt_iter]
        if qt_iter == 0: holdings = quantile_1
        df_1 = ln_.copy()
        for i in range(len(quantile_1)):
            df_1.loc[index[i], ~df_1.columns.isin(quantile_1[i])] = 0
        weighting_1 = -_reweighting_equal(df_1)
        ret = pct_calculating_profit.loc[weighting_1.index[0]:] * weighting_1
        if type_ == "cumsum":
            quantiles[qt_iter] = ret.sum(axis=1).cumsum()
        else:
            quantiles[qt_iter] = (1 + ret.sum(axis=1)).cumprod() - 1
    quantilized_cumsum = pd.DataFrame(quantiles).T
    fig = go.Figure()
    for i in range(len(quantilized_cumsum.columns)):
        fig.add_trace(go.Scatter(
            x=quantilized_cumsum.index,
            y=quantilized_cumsum.iloc[:, i],
            name=f"{i} th quantile",
            mode="lines",
            # line=dict(color="orange"),
            yaxis="y1"
        ))
    fig.show()
    # quantilized_cumsum.iloc[:, :].plot(title="10 quantile cumsum distribution")
    return holdings

In [4]:
from ibapi.client import *
from ibapi.wrapper import *
import datetime
import time
import threading


from ibapi.contract import Contract
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
import psycopg2
from time import sleep
import pandas as pd
port = 7496

class TestApp(EClient, EWrapper):
    def __init__(self):
        EClient.__init__(self, self)
        self.orderId = 0
        self.list_ = []
        etf = pd.read_csv("hk_list.csv")
        self.etfList = list(etf.iloc[:, 2])
        self.index = 0
        self.curCode = self.etfList[self.index]
        # self.tw_maincode_offshore_list = 
    def nextValidId(self, orderId):
        self.orderId = orderId
    def nextId(self):
        self.orderId += 1
        return self.orderId

    def error(self, reqId, errorCode, errorString, advancedOrderReject=""):
        print(f"reqId: {reqId}, errorCode: {errorCode}, errorString: {errorString}, orderReject: {advancedOrderReject}")

    def historicalData(self, reqId, bar):
        bar = bar.to_dict()
        bar['symbol'] = self.curCode
        self.list_.append(bar)
    def historicalDataEnd(self, reqId, start, end):
        self.cancelHistoricalData(reqId)
        self.index += 1
        self.curCode = self.etfList[self.index]
    def loop(self):
        for i in self.etfList:
            mycontract = Contract()
            mycontract.symbol = i
            mycontract.secType = "STK"
            mycontract.exchange = "SMART"
            mycontract.currency = "USD"
            nextId = app.nextId()
            app.reqHistoricalData(nextId, mycontract, "20250124 09:00:00 US/Eastern", "1 Y", "1 day", "TRADES", 1, 1, False, [])
app = TestApp()
app.connect("127.0.0.1", port, 0)
threading.Thread(target=app.run).start()
time.sleep(1)

reqId: -1, errorCode: 2104, errorString: Market data farm connection is OK:usfarm.nj, orderReject: 
reqId: -1, errorCode: 2104, errorString: Market data farm connection is OK:usfuture, orderReject: 
reqId: -1, errorCode: 2104, errorString: Market data farm connection is OK:cashfarm, orderReject: 
reqId: -1, errorCode: 2104, errorString: Market data farm connection is OK:usopt, orderReject: 
reqId: -1, errorCode: 2104, errorString: Market data farm connection is OK:usfarm, orderReject: 
reqId: -1, errorCode: 2106, errorString: HMDS data farm connection is OK:ushmds, orderReject: 
reqId: -1, errorCode: 2158, errorString: Sec-def data farm connection is OK:secdefnj, orderReject: 


reqId: 2, errorCode: 200, errorString: No security definition has been found for the request, orderReject: 
reqId: 3, errorCode: 200, errorString: No security definition has been found for the request, orderReject: 
reqId: 4, errorCode: 200, errorString: No security definition has been found for the request, orderReject: 
reqId: 5, errorCode: 200, errorString: The destination or exchange selected is Invalid. Please review your order's "Destination" field. If using a <br>Directed order, review the exchange selected when creating the order ticket or order row. This may occur when <br>creating stock orders for the overnight session or when creating option orders for the overnight session., orderReject: 
reqId: 6, errorCode: 200, errorString: The destination or exchange selected is Invalid. Please review your order's "Destination" field. If using a <br>Directed order, review the exchange selected when creating the order ticket or order row. This may occur when <br>creating stock orders for

In [19]:
mycontract = Contract()
mycontract.symbol = "MCD"
mycontract.secType = "STK"
mycontract.exchange = "SMART"
mycontract.currency = "USD"
nextId = app.nextId()
app.reqHistoricalData(nextId, mycontract, "20250124 09:00:00 US/Eastern", "1 Y", "1 day", "TRADES", 1, 1, False, [])
